In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from random import choice
import random
from random import randrange
import collections

def most_character_number(one_string):
    static_result = collections.Counter(one_string) 
    result = dict(static_result)                            
    most_number = max([value for value in result.values()])
    most_character = [key for key,value in result.items() if value==most_number]
    return (most_number,most_character)

def load(filename):
    f = open(filename, 'r')
    text = f.read()
    if(filename == 'symbols.txt'):
        text = text.replace('\n','')
    elif (filename == 'warandpeace.txt'):
        text = text.replace('\n',' ')
        text = text.replace('“','"')
        text = text.replace('”','"')
        text = text.replace('‘','\'')
        text = text.replace('’','\'')
        text = text.replace('á','a')
        text = text.replace('é','e')
        text = text.replace('ë','e')
        text = text.replace('í','i')
        text = text.replace('ó','o')
        text = text.replace('#','')
        text = text.replace('à','a')
        text = text.replace('ú','u')
        text = text.replace('è','e')
        text = text.replace('—','-')
        text = text.replace('î','i')
        text = text.replace('ô','o')
        text = text.replace('ê','e')
        text = text.replace('ï','i')
        text = text.replace('ç','c')
        text = text.replace('%','')
        text = text.replace('@','')
              
    f.close()
    return text

def learnTrans_matrix(text,symbols):
    trans_matrix =np.ones((len(symbols),len(symbols)))  #from 0
    position =0
    while position+1 != len(text):  #position from 0
        current_letter = text[position].lower()
        next_letter = text[position+1].lower()
        
        current_position = symbols.find(current_letter)
        next_position = symbols.find(next_letter)
        
        if current_position == -1 or next_position == -1:
            position = position+1
        else:
            trans_matrix[current_position, next_position] =  trans_matrix[current_position, next_position] + 1
            position = position+1
            
    for i in range(0, len(symbols)-1):
        trans_matrix[i,:] = trans_matrix[i,:]/np.sum(trans_matrix[i,:])
        
    return trans_matrix
    
    
def hinton(matrix, max_weight=None, ax=None):
    """Draw Hinton diagram for visualizing a weight matrix."""
    ax = ax if ax is not None else plt.gca()

    if not max_weight:
        max_weight = 2 ** np.ceil(np.log(np.abs(matrix).max()) / np.log(2))

    ax.patch.set_facecolor('gray')
    ax.set_aspect('equal', 'box')
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())

    for (x, y), w in np.ndenumerate(matrix):
        color = 'white' if w > 0 else 'black'
        size = np.sqrt(np.abs(w) / max_weight)
        rect = plt.Rectangle([x - size / 2, y - size / 2], size, size,
                             facecolor=color, edgecolor=color)
        ax.add_patch(rect)

    ax.autoscale_view()
    ax.invert_yaxis()
    
def mapping(keys, message):
    item =0
    mappingTxt =''
    for item in range(len(message)-1): 
        index = symbols.find(message[item])
        mappingTxt = ''.join((mappingTxt,keys[index]))
    return mappingTxt    

def swap(keys,X):
    i = choice(X)
    j = choice(X)
    while i==j:
        j = choice(X)
    keys = list(keys)
    keys[i], keys[j] = keys[j], keys[i]
    return keys
    
def MHdecrypt(filename, iteration=50000):
    message = load(filename)
    message = message.lower()
    keys = searchKey(message, iteration)
    print(keys)
    true_message = mapping(keys,message)
    print(true_message)
 
    
def searchKey(message, iteration):
    keys = symbols
    best_keys = ''
    best_score = -1000000000
    [number,charr]=most_character_number(message)
    char = charr[0]
    index = keys.find(char)
    index_space = keys.find(' ')
    keys = list(keys)
    keys[index] = ' '
    keys[index_space] = char
    
    X = np.arange(0, 53, 1)
    X= np.delete(X,index)
    convery_keys =''
    for i in range(iteration):
        current_keys = keys
        new_keys = swap(keys, X)
        new_score = score(new_keys,message)
        current_score = score(current_keys,message)

        alpha = min(1, np.exp(new_score-current_score))
            
        u = random.uniform(0,1)  
       
        if new_score > current_score:
            if  u < alpha:
                keys = new_keys
                best_score =new_score
                best_keys = new_keys
            #print(best_score)
                
        if (i>=500) and (i%500==0):
            print('Iteration')
            print(i)
            print(best_keys[0:52])
            mappingText = mapping(best_keys,message)
            print(mappingText)
            #print(best_score)
            print(current_score, new_score)
        if (i>=2000) and (i%2000==0):
            last_keys = convery_keys
            convery_keys = best_keys
            if (last_keys !=''):
                for k in range(len(best_keys)-1):
                    if convery_keys[k] == last_keys[k]:
                        index = np.argwhere(X==k)
                        X=np.delete(X,index)
    #best_keys = keys                     
    return best_keys
            
        
def score(keys,message):
    mappingTxt = mapping(keys, message)
    score = 0
    position = 0
    for position in range(len(mappingTxt)-1):
        current_letter =  mappingTxt[position]
        next_letter = mappingTxt[position+1]
        current_index = symbols.find(current_letter)
        next_index = symbols.find(next_letter)
        prob = trans_matrix[current_index,next_index]
        score = score +  np.log(prob)
    return score



#---------------------------------------------
symbols = load('symbols.txt')
training_text = load('warandpeace.txt')
trans_matrix = learnTrans_matrix(training_text,symbols)

"""
plt.figure(figsize=(10,10))
X=np.arange(0,53,1)
Y=np.arange(0,53,1)
X_ticks = ['=',' ','-',',',';',':','!','?','/','.','&','"','(',')','[',']','*','0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
Y_ticks = ['=',' ','-',',',';',':','!','?','/','.','&','"','(',')','[',']','*','0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
plt.xticks(X, X_ticks)
plt.yticks(Y, Y_ticks)
plt.pcolor(X,Y,trans_matrix,cmap='binary');
plt.colorbar()
plt.grid(True)
plt.show()

plt.figure(figsize=(10,10))
hinton(trans_matrix)
plt.show()
"""
#-------------------------------------------------------------------
#MCMC decrypting


MHdecrypt('message.txt')




Iteration
500
['f', '(', 'h', ',', 'o', 'e', 'l', 'r', 'b', 'm', '7', 'q', ']', ')', 'z', 'i', '1', '-', 'u', 't', '3', '=', '.', '*', "'", '8', 'k', '9', 'x', '0', '5', '4', '/', 'd', ';', ':', 'j', '[', 'w', 'c', 'y', '!', ' ', '?', 'p', 's', '2', 'v', '"', 'a', 'n', '6']
ze uw wineprl aey uilr cnoerlamor wrald uw bashrl pacr ur diur aycztr shas z(cr mrre snlezep icrl ze uw uzey rcrl dzetr. *ghrercrl win brro oz'r tlzsztz-zep aew ier,* hr sioy ur, *"nds lrurumrl shas aoo shr vrivor ze shzd giloy hacre(s hay shr aycaesaprd shas win(cr hay.* hr yzye(s daw aew uilr mns gr(cr aogawd mrre nendnaoow tiuuneztaszcr ze a lrdrlcry gaw, aey z neyrldsiiy shas hr uraes a plras yrao uilr shae shas. ze tiedrknretr z(u zetozery si lrdrlcr aoo "nypuresd, a hamzs shas had ivrery nv uaew tnlzind easnlrd si ur aey aodi uayr ur shr cztszu ib eis a brg crsrlae milrd. shr ameiluao uzey zd knzt' si yrsrts aey assath zsdrob si shzd knaozsw ghre zs avvrald ze a eiluao vrldie, aey di zs taur amins shas ze tioo

Iteration
3000
['(', "'", 'h', ',', 'l', 's', 'r', 'e', 'k', 'b', '7', 'q', ']', '?', 'i', 'o', '1', 'x', 'c', 'u', '3', '!', '.', '"', 'g', '8', 'v', '9', '*', '6', '5', '4', '=', 'd', ';', '/', 'j', '[', 'y', 'm', 't', ':', ' ', ')', 'p', 'z', '2', 'f', '-', 'a', 'n', '0']
is cy yonsper ast core mnlserable yeard cy kazher pame ce doce atmiue zhaz i'me bees znrsisp omer is cy cist emer disue. "whesemer yon keel lige uriziuixisp asy ose," he zolt ce, "-ndz rececber zhaz all zhe feofle is zhid worlt hames'z hat zhe atmaszaped zhaz yon'me hat." he tits'z day asy core bnz we'me alwayd bees nsndnally uoccnsiuazime is a redermet way, ast i nsterdzoot zhaz he ceasz a preaz teal core zhas zhaz. is uosdevnesue i'c isuliset zo rederme all -ntpceszd, a habiz zhaz had ofeset nf casy unriond saznred zo ce ast aldo cate ce zhe miuzic ok soz a kew mezeras bored. zhe absorcal cist id vniug zo tezeuz ast azzauh izdelk zo zhid vnalizy whes iz affeard is a sorcal ferdos, ast do iz uace abonz zhaz is uol

Iteration
5500
['(', "'", 'h', ',', 'l', 'n', 'r', 'e', 'f', 'b', '7', 'q', ']', '?', 'i', 'o', '1', 'x', 'm', 'c', '3', '!', '.', '"', 'v', '8', 'j', '9', '*', '-', '5', '4', '=', 't', ';', '6', '/', '[', 'y', 's', 'd', ':', ' ', ')', 'g', 'z', '2', 'p', 'k', 'a', 'u', '0']
in my younger and more sulnerable yeart my fazher gase me tome adsice zhaz i'se been zurning oser in my mind eser tince. "wheneser you feel live crizicixing any one," he zold me, "kutz remember zhaz all zhe people in zhit world hasen'z had zhe adsanzaget zhaz you'se had." he didn'z tay any more buz we'se alwayt been unutually communicazise in a retersed way, and i undertzood zhaz he meanz a greaz deal more zhan zhaz. in contejuence i'm inclined zo reterse all kudgmenzt, a habiz zhaz hat opened up many curiout nazuret zo me and alto made me zhe siczim of noz a few sezeran boret. zhe abnormal mind it juicv zo dezecz and azzach iztelf zo zhit jualizy when iz appeart in a normal perton, and to iz came abouz zhaz in col

Iteration
8000
['(', "'", 'h', ',', 'l', 'n', 'r', 'e', 'f', 'b', '7', 'q', ']', '?', 'i', 'o', '1', 'x', 'm', 'c', '3', '!', '.', '"', 'v', '8', 'j', '9', '*', '-', '5', '4', '=', 't', ';', '6', '/', '[', 'y', 's', 'd', ':', ' ', ')', 'g', 'z', '2', 'p', 'k', 'a', 'u', '0']
in my younger and more sulnerable yeart my fazher gase me tome adsice zhaz i'se been zurning oser in my mind eser tince. "wheneser you feel live crizicixing any one," he zold me, "kutz remember zhaz all zhe people in zhit world hasen'z had zhe adsanzaget zhaz you'se had." he didn'z tay any more buz we'se alwayt been unutually communicazise in a retersed way, and i undertzood zhaz he meanz a greaz deal more zhan zhaz. in contejuence i'm inclined zo reterse all kudgmenzt, a habiz zhaz hat opened up many curiout nazuret zo me and alto made me zhe siczim of noz a few sezeran boret. zhe abnormal mind it juicv zo dezecz and azzach iztelf zo zhit jualizy when iz appeart in a normal perton, and to iz came abouz zhaz in col